In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import model_from_json
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



pd.options.display.max_columns = None
pd.options.display.max_rows = 100



Using TensorFlow backend.


In [2]:
df = pd.read_csv('Data/kc_house_data.csv').drop('Unnamed: 0', axis = 1).dropna().sort_values(by= 'date', ascending = False)
df['price'] = df.price.map(lambda x: x/1000000)
df.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
4542,7201800300,9/9/2014,0.3975,3,1.75,1300,8480,1.0,0.0,0.0,3,7,1300,0.0,1969,0.0,98052,47.6991,-122.130,1740,7280
7888,5470100220,9/9/2014,0.2220,3,1.50,1310,9273,1.0,0.0,0.0,4,7,1310,0.0,1968,0.0,98042,47.3683,-122.147,1710,9600
14566,9528104910,9/9/2014,0.7960,4,3.25,2110,3000,2.0,0.0,0.0,3,8,2110,0.0,2001,0.0,98115,47.6769,-122.328,1780,4000


In [3]:
random_state = 10 



standard = StandardScaler()
min_max = MinMaxScaler()
ignore_col = 'id date price'.split()
#x = df[['bathrooms', 'sqft_living', 'grade', 'sqft_above', 'sqft_living15']]
x = df[[i for i in df.columns if i not in ignore_col]]
y = df[['price']]

x = standard.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = .8, random_state = random_state)

In [9]:
def sequential_AAN(x,y, metrics = ['mae']): 
    drop = .2

    model = Sequential()
    
    model.add(Dense(128,input_dim = x.shape[1], activation = 'linear'))
    model.add(Dropout(drop)) #regularization

    
    model.add(Dense(64 , activation = 'linear'))
    model.add(Dropout(drop))

    model.add(Dense(32, activation = 'linear'))
    model.add(Dropout(drop)) 
    
#     model.add(Dense(128, activation = 'relu'))
#     model.add(Dropout(drop)) 

    
    model.add(Dense(1))
    model.compile(loss = 'mse', optimizer = 'sgd', metrics = metrics)
    return model

In [6]:
epochs = 15
seq_model = sequential_AAN(x_train, y_train)
seq_history = seq_model.fit(x_train, y_train, epochs = epochs, batch_size = 16, validation_data = (x_test, y_test))

Epoch 1/15
789/789 [==============================] - 2s 2ms/step - loss: 0.0917 - mae: 0.1962 - val_loss: 0.0470 - val_mae: 0.1317
Epoch 2/15
789/789 [==============================] - 1s 2ms/step - loss: 0.0590 - mae: 0.1534 - val_loss: 0.0454 - val_mae: 0.1279
Epoch 3/15
789/789 [==============================] - 1s 2ms/step - loss: 0.0511 - mae: 0.1434 - val_loss: 0.0379 - val_mae: 0.1194
Epoch 4/15
789/789 [==============================] - 1s 1ms/step - loss: 0.0503 - mae: 0.1367 - val_loss: 0.0383 - val_mae: 0.1185
Epoch 5/15
789/789 [==============================] - 1s 2ms/step - loss: 0.0443 - mae: 0.1327 - val_loss: 0.0336 - val_mae: 0.1129
Epoch 6/15
789/789 [==============================] - 1s 2ms/step - loss: 0.0441 - mae: 0.1318 - val_loss: 0.0304 - val_mae: 0.1116
Epoch 7/15
789/789 [==============================] - 1s 2ms/step - loss: 0.0424 - mae: 0.1291 - val_loss: 0.0529 - val_mae: 0.1305
Epoch 8/15
789/789 [==============================] - 1s 1ms/step - loss: 0.

In [7]:
# json = open('models/ANN_Model.json', 'r')
# ann_json = json.read()
# json.close()
# seq_ann = model_from_json(ann_json)
# seq_ann.load_weights('models/ANN_Model_Weights.h5')

In [8]:
ann_json = seq_model.to_json()
with open('Models/ANN_Model.json', 'w') as file:
    file.write(ann_json)
    
seq_model.save('Models/ANN_Model_Weights.h5')

In [12]:
seq_history['history']['loss']

TypeError: 'History' object is not subscriptable